# 3.Merging The IMDB and 'the-numbers' Tables
In this notebook we will unify the data from both sets we created in steps 1 and 2.

In [1]:
import os
import subprocess
import pandas as pd
import numpy as np
from hashlib import md5 

### 3.1 Loading the data sets
We will load the csv files of both sets into dataframes.
We will also modify 'the-numbers' table to have a column ready for the tconst values of the 'imdb' table.

In [2]:
imdb_df = pd.read_csv("output_data/fixed.title.basics.csv")
thenumbers_df = pd.read_csv("output_data/the_numbers.csv")
thenumbers_df.insert(0,"tconst","",allow_duplicates=False)
thenumbers_df.insert(5,"genres","",allow_duplicates=True)
thenumbers_df.insert(5,"runtime","",allow_duplicates=True)
thenumbers_df.insert(7,"year","",allow_duplicates=True)
thenumbers_df.insert(7,"month","",allow_duplicates=True)
thenumbers_df.insert(7,"day","",allow_duplicates=True)
thenumbers_df.genre=""
thenumbers_df.tconst=""
thenumbers_df.date=thenumbers_df.date.fillna("0000/00/00")

df2 = pd.DataFrame(thenumbers_df.date.str.split("/").tolist(),columns=["year","month","day"])
thenumbers_df[["year","month","day"]]=df2[["year","month","day"]]\

#The csv imports an unnecessary column of the old indicies before the sort by date, we will delete it.abs
thenumbers_df=thenumbers_df.drop("Unnamed: 0",1)
thenumbers_df=thenumbers_df.drop("date",1)


In [3]:
thenumbers_df

,tconst,id,movie,link,runtime,genres,day,month,year,dist,...,tickets_2011,tickets_2012,tickets_2013,tickets_2014,tickets_2015,tickets_2016,tickets_2017,tickets_2018,tickets_2019,tickets_2020
0,,1ce88cd3a75300f44f56511e82ab4b12,Moana,/movie/Moana-(Documentary)-(1926),,,07,01,1926,Paramount Pictures,...,NaN,NaN,NaN,NaN,NaN,346.0,NaN,NaN,NaN,NaN
1,,767146afaa42d85e4265992656f9d079,Metropolis,/movie/Metropolis-(Germany)-(1927),,,13,03,1927,Paramount Pictures,...,70165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,3d54ed959d3c0916442ecc7a27361f18,La Grande Illusion,/movie/Grande-Illusion-La,,,12,09,1938,Rialto Pictures,...,NaN,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,4cf8e15b66f14b1571979dc69089867d,Arabian Nights,/movie/Arabian-Nights,,,25,12,1942,Universal,...,NaN,NaN,NaN,NaN,NaN,429.0,NaN,NaN,NaN,NaN
4,,020037917f805b5e80b9852a6272eb93,Detour,/movie/Detour-(1945),,,30,11,1945,Producers Releasi…,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1606.0,168.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13621,,f91f05be3692e449472e88cbac05a233,Ta-weo,/movie/Ta-Weo,,,00,00,0000,NaN,...,NaN,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13622,,c4946644789c203efe43e063613433c4,The ABCs of Death,/movie/ABCs-of-Death-The,,,00,00,0000,NaN,...,NaN,NaN,2685.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13623,,ff8b957fb62af22108f5ad7e3e83e70e,Next Goal Wins,/movie/Next-Goal-Wins,,,00,00,0000,NaN,...,NaN,NaN,NaN,5274.0,NaN,NaN,NaN,NaN,NaN,NaN
13624,,6859dd3b4dbdfe4cff8d646887b1037d,Shadow in the Cloud,/movie/Shadow-in-the-Cloud-(2020-New-Zealand),,,00,00,0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4270.0


## 3.2 Addressing the problems...
First of all, to merge the tables we need to have a strong link to more data on imdb. To have that link we can use the ``tconst`` field in the table.
It is a unique key tied to the movie.

The problem is linking that key to the matching movie in the 'the-numbers' table. The only way we can do it is with string comparison.
This alone raises another problem of duplicate movie names, for example the movie name 'Moana' appreas twice as shown below.


In [4]:
thenumbers_df[thenumbers_df.movie=="Moana"]

,tconst,id,movie,link,runtime,genres,day,month,year,dist,...,tickets_2011,tickets_2012,tickets_2013,tickets_2014,tickets_2015,tickets_2016,tickets_2017,tickets_2018,tickets_2019,tickets_2020
0,,1ce88cd3a75300f44f56511e82ab4b12,Moana,/movie/Moana-(Documentary)-(1926),,,07,01,1926,Paramount Pictures,...,NaN,NaN,NaN,NaN,NaN,346.0,NaN,NaN,NaN,NaN
11503,,f81d7ee53f44d3fb965f704725b206cb,Moana,/movie/Moana-(2016),,,23,11,2016,Walt Disney,...,NaN,NaN,NaN,NaN,NaN,24282787.0,4315599.0,NaN,NaN,NaN


To solve the this problem we will try to find movies in IMDB table that match movies in 'the-numbers' table and we will compare the release years to pinpoint the exact movies.
If that alone isn't enough (meaning same name of movies at the same year) we will then crawl into 'the-numbers' to the movie's page and compare the runtime.
<br><br>
Now, if it isn't enough there is another _technical_ problem. We need to compare around 13K strings to about 320K strings, this on the CPU will take hours to run.
To solve this problem we have written a software that compares hashes on the GPU parallely amplifying the calculation several hundreds if not thousands of times.
<br><br>
This software was compiled with nvcc CUDA compiler and we will use that software with subproccess calls to get results

## 3.2 Cut Movie Names Problem
We noticed that in 'the-numbers' table some movie names are cut and end with the special char `…`. This is the case everytime the movie has more than 27 characters, therfore we will cut every movie name in both tables to be exactly at most 27 chars.

## 3.2 Hashing the names
We will now create two files each for each table names hashed with MD5.
This step is crucial to do parallel comparsion of fixed sized arrays to cut massive calculation time.
NOTE: we also take care of the length problem.

Also also we will create two hashed versions of the names, one for `primaryTitle` colunmn and one for `originalTitle` and we will try to find matches on the `primaryTitle` first and if there are still mismatches we will try then `originalTitle`.

In [5]:
cuda_data_dir = "cuda_data/"
primary_path = cuda_data_dir+"primaryTitle"
original_path = cuda_data_dir+"originalTitle"
numbers_path = cuda_data_dir+"the_numbers"
def hash_and_save_movie(file,movie:str):
    name = movie[0:27]
    name_hash = md5(movie.lower().encode("utf-8")).hexdigest()
    file.write(name_hash)

In [6]:

#save priamryTItle hases
with open(primary_path,"w") as f:
    for index,row in imdb_df.iterrows():
        hash_and_save_movie(f,row.primaryTitle)
    f.close()

#save originalTitle hases
with open(original_path,"w") as f:
    for index,row in imdb_df.iterrows():
        hash_and_save_movie(f,row.originalTitle)
    f.close()
    
#save the-numbers hashes
with open(numbers_path,"w") as f:
    for index,row in thenumbers_df.iterrows():
        hash_and_save_movie(f,row.movie)
    f.close()



## Matching the Names
We use our specially crafted tool using the GPU to compare each hash of each table with one another. The tool "cudamerge" will provide an output file that consists of rows where each row represents 'the-numbers' rows and contains all matching movies in the 'imdb' table, where the match is the index of the table.

In [7]:
cudamerge_path = cuda_data_dir+"cudamerge"
output_primary_path = cuda_data_dir+"outputPrimaryMatches"
output_original_path = cuda_data_dir+"outputOriginalMatches"
subprocess.call([cudamerge_path,primary_path,f"{imdb_df.shape[0]}",numbers_path,f"{thenumbers_df.shape[0]}",output_primary_path])
subprocess.call([cudamerge_path,original_path,f"{imdb_df.shape[0]}",numbers_path,f"{thenumbers_df.shape[0]}",output_original_path])

0

## Importing the output.
We will import the output of both original and primary titles and put it in a dictionary where the key is the index of 'the-numbers' table and the value is a list of all matching indicies in 'imdb' table.<br/>
We will also skip empty lists (where there is no matching name in IMDB to the-numbers).

In [8]:
def createMatchingEntriesDict(output_path:str):
    matching_entries={}
    with open(output_path) as file:
        rows = file.readlines()
        for index in range(len(rows)):
            entries = rows[index].split(" ")[:-1]
            if len(entries)!=0:
                matching_entries[index]= entries 
    return matching_entries

In [9]:
matching_entries=createMatchingEntriesDict(output_primary_path)
matching_original_entries=createMatchingEntriesDict(output_original_path)
for index,match in matching_original_entries.items():
    try:
        matching_entries[index]
    except:
        matching_entries[index]=match

## Concluding The Matches
We have successfully matched 11687 entries from a total of 13625 which is 86% matching. We are satisfied with the results and now can continue merging the data.

In [10]:
len(matching_entries)/thenumbers_df.shape[0]

0.8578452957581095

## Single Matching entries
We will start with assigning a 'tconst' value for the indicies in the-numbers that have exactly 1 match.<br/>
After the assignment, we will delete this entry (in a copied dict) and then update the main dict again

In [11]:
def copyData(thenumbers_index:int,imdb_index:str):
    imdb_index = int(imdb_index)
    thenumbers_df.at[thenumbers_index,"tconst"] = imdb_df.at[imdb_index,"tconst"]
    thenumbers_df.at[thenumbers_index,"genres"] = imdb_df.at[imdb_index,"genres"]
    thenumbers_df.at[thenumbers_index,"runtime"] = imdb_df.at[imdb_index,"runtimeMinutes"]
    if int(thenumbers_df.at[thenumbers_index,"year"])==0:
        thenumbers_df.at[thenumbers_index,"year"]=imdb_df.at[imdb_index,"startYear"]


In [12]:
matching_entries_copy = {**matching_entries}
for index, matchlist  in matching_entries.items():
    if len(matchlist)==1:
        copyData(index,matchlist[0])
        del matching_entries_copy[index]

matching_entries={**matching_entries_copy}
del matching_entries_copy

After this step we are left with 3325 multiple matches

In [13]:
len(matching_entries)

3325

## Matching same year movies
Now we will try to match movies that were released in the same year (and continue handling for multiple movies)

In [14]:
matching_entries_copy = {**matching_entries}
same_year_duplicates = {}
no_match_duplicates={}
for index, entries in matching_entries.items():
    try:
        year = int(thenumbers_df.at[index,'year'])
        tmp_entries=[]
        for entry in entries:
            entry = int(entry)
            if abs(year - imdb_df.at[entry,'startYear'])<=1:
                tmp_entries.append(entry)
        if len(tmp_entries) == 0:
            no_match_duplicates[index]=entries
        if len(tmp_entries)>1:
            same_year_duplicates[index]=tmp_entries
        if len(tmp_entries) == 1:
            copyData(index,tmp_entries[0])
        del matching_entries_copy[index]
    except: pass
matching_entries={**matching_entries_copy}
del matching_entries_copy

## Manual Matching
After the automatic matching, we are left with many results that require manual handling because various reasos such as domestic vs worldwide release or movie names mismatch...
Due to time limits we will not refine the matching further and we are left with the results above as the final dataset.

Here a a few examples of how we would have done it, but there are almost 500 results that require manual handling...

In [15]:
def printIMDBDetails(loc:int):
    row = imdb_df.iloc[loc]
    print(f"{row.tconst}\t | {row.primaryTitle} | {row.originalTitle} | {row.startYear} | {row.runtimeMinutes}")
def printTheNumbersDetails(loc:int):
    row = thenumbers_df.iloc[loc]
    print(f"[{loc}] | {row.movie} | {row.day}/{row.month}/{row.year} | {row.dist} | {row.link}")

data={ }

for index,values in data.items():
    printTheNumbersDetails(index)
    
    for v in values:
        printIMDBDetails(int(v))
    print("\n")


In [16]:
#fixing data

#Leon 

fixes = {
    13:"tt0042803",
    43:"tt0063794",
    88:"tt0082307",
    151:"tt0110413",
    158:"tt0111495",
    292:"tt0110877"
    }

for index, tconst in fixes.items():
    try:
        del no_match_duplicates[index]
        thenumbers_df.at[index,"tconst"]=tconst
    except:
        #print(index)
        pass

In [17]:
len(no_match_duplicates)

311

## Assigning IMDB's Genres to the-numbers
The genres in IMDB are more specific so we'll use them
we will also drop the hashed id because we have the tconst values refering to IMDB.

In [18]:
thenumbers_df=thenumbers_df.drop("id",1)
thenumbers_df=thenumbers_df[thenumbers_df.tconst!=""]

In [19]:
thenumbers_df

,tconst,movie,link,runtime,genres,day,month,year,dist,gross_1995,...,tickets_2011,tickets_2012,tickets_2013,tickets_2014,tickets_2015,tickets_2016,tickets_2017,tickets_2018,tickets_2019,tickets_2020
0,tt0017162,Moana,/movie/Moana-(Documentary)-(1926),77,Documentary,07,01,1926,Paramount Pictures,NaN,...,NaN,NaN,NaN,NaN,NaN,346.0,NaN,NaN,NaN,NaN
1,tt0017136,Metropolis,/movie/Metropolis-(Germany)-(1927),153,"Drama,Sci-Fi",13,03,1927,Paramount Pictures,NaN,...,70165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0028950,La Grande Illusion,/movie/Grande-Illusion-La,113,"Drama,War",12,09,1938,Rialto Pictures,NaN,...,NaN,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0034465,Arabian Nights,/movie/Arabian-Nights,86,"Action,Adventure,Comedy",25,12,1942,Universal,NaN,...,NaN,NaN,NaN,NaN,NaN,429.0,NaN,NaN,NaN,NaN
4,tt0037638,Detour,/movie/Detour-(1945),68,"Crime,Drama,Film-Noir",30,11,1945,Producers Releasi…,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1606.0,168.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13621,tt2554270,Ta-weo,/movie/Ta-Weo,121,"Action,Drama,Thriller",00,00,2012,NaN,NaN,...,NaN,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13622,tt1935896,The ABCs of Death,/movie/ABCs-of-Death-The,129,"Comedy,Horror",00,00,2012,NaN,NaN,...,NaN,NaN,2685.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13623,tt2446600,Next Goal Wins,/movie/Next-Goal-Wins,97,"Documentary,Sport",00,00,2014,NaN,NaN,...,NaN,NaN,NaN,5274.0,NaN,NaN,NaN,NaN,NaN,NaN
13624,tt9691136,Shadow in the Cloud,/movie/Shadow-in-the-Cloud-(2020-New-Zealand),83,"Action,Horror,War",00,00,2020,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4270.0


## Concluding the merge
We are finally left with the last table we will use further on. This table consists of the tconst and genres from IMDB and has the grossing info from 'the-numbers' website.
We will save it and on the next notebook open it to learn some information about it visually.

In [20]:
thenumbers_df.to_csv("./output_data/final_table.csv")